# Exporting to a Pandas DataFrame

The traces captured with `myhdlpeek` can be exported to Pandas Data Frames.
Once again, I'll demonstrate using the multiplexer example:

In [1]:
from myhdl import *
from myhdlpeek import Peeker  # Import the myhdlpeeker module.

def mux(z, a, b, sel):
    """A simple multiplexer."""
    @always_comb
    def mux_logic():
        if sel == 1:
            z.next = a  # Signal a sent to mux output when sel is high.
        else:
            z.next = b  # Signal b sent to mux output when sel is low.
    return mux_logic

# Create some signals to attach to the multiplexer.
a, b, z = [Signal(0) for _ in range(3)]  # Integer signals for the inputs & output.
sel = Signal(intbv(0)[1:])               # Binary signal for the selector.

# Create some Peekers to monitor the multiplexer I/Os.
Peeker.clear()         # Clear any existing Peekers. (Start with a clean slate.)
Peeker(a, 'a')         # Add a Peeker to the a input.
Peeker(b, 'b')         # Add a Peeker to the b input.
Peeker(z, 'z')         # Add a peeker to the z output.
Peeker(sel, 'select')  # Add a Peeker to the select input. The Peeker label doesn't have to match the signal name.

# Instantiate mux.
mux_1 = mux(z, a, b, sel)

# Apply random patterns to the multiplexer.
from random import randrange
def test():
    for _ in range(8):
        a.next, b.next, sel.next = randrange(8), randrange(8), randrange(2)
        yield delay(1)

# Simulate the multiplexer, testbed and the peekers.
sim = Simulation(mux_1, test(), *Peeker.instances()).run()

<class 'myhdl.StopSimulation'>: No more events


Once the simulation has run, I can export the results to a Pandas DataFrame:

In [2]:
Peeker.to_dataframe()

,a,b,select[0],z[0]
0,0,0,0,0
1,7,1,0,1
2,6,7,0,7
4,2,7,1,2
5,3,4,1,3
6,7,6,1,7
7,6,0,0,0


The same options that were used for tabular output are available for exporting the DataFrame: 

In [3]:
Peeker.to_dataframe('a b z', start_time=3, stop_time=6) # Select the traces to export and set the start and stop times.

,a,b,z
3,6,7,7
4,2,7,2
5,3,4,3
6,7,6,7
